In [195]:
import pandas as pd
import pytesseract
from pytesseract import Output
import cv2
import json
from pdf2image import convert_from_path
import numpy as np

os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/Cellar/tesseract/5.2.0/share/tessdata'
multipliersDict = json.load(open('multipliersDict.json'))


In [208]:
#Converting PDF's to PNG

for file in os.listdir('Images'):
  if file.split('.')[-1]=='pdf':
    print(file)
    pdfFile = convert_from_path(f'images/{file}',dpi=400)
    
    for i in range(len(pdfFile)):
        # Save pages as images in the pdf
        pdfFile[i].save(f'PNGs/{file.split(".")[0]}_page{i}.png', 'PNG')

FGS8B39_2022.pdf
DKR8842_2022.pdf
EBB4B49_2022.pdf
ENU2816.pdf
DOC EEE4618.pdf
ESA2554.pdf
OAW9991 - CAVALO T1.pdf
ECT2365_2022.pdf


In [ ]:
filename = 'DKR8842_2022.pdf_page0.png'
img = cv2.imread(f'PNGs/{filename}') 
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,img = cv2.threshold(img,240,255,cv2.THRESH_BINARY)
results = pytesseract.image_to_data(img, output_type=Output.DICT,lang='por')
img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)

In [201]:
filename = 'DKR8842_2022.pdf_page0.png'
img = cv2.imread(f'PNGs/{filename}') 
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,img = cv2.threshold(img,240,255,cv2.THRESH_BINARY)
results = pytesseract.image_to_data(img, output_type=Output.DICT,lang='por')

img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)


keywords = ['COMBUSTÍVEL', 'LOTAÇÃO','CPF','CNPJ']

for i in range(0, len(results["text"])):
     if results['text'][i] in keywords:
        print(results['text'][i])
        # We can then extract the bounding box coordinates
        # of the text region from  the current result
        x = int(results["left"][i])
        y = int(results["top"][i])
        w = int(results["width"][i])
        h = int(results["height"][i])

        # We will also extract the OCR text itself along
        # with the confidence of the text localization
        text = results["text"][i]
        conf = int(results["conf"][i])
        page_num = results["page_num"][i]
        block_num = results["block_num"][i]
        par_num = results["par_num"][i]
        line_num = results["line_num"][i]
        word_num = results["word_num"][i]
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 0, 255), 10)

        # if conf > 0:
            
            # We will display the confidence and text to
            # our terminal
            # print(f"Confidence: {conf}")
            # print(f"Text: {text}")
            # print(f"page_num: {page_num}")
            # print(f"block_num: {block_num}")
            # print(f"par_num: {par_num}")
            # print(f"line_num: {line_num}")
            # print(f"word_num: {word_num}")
            # print("")

            
            # We then strip out non-ASCII text so we can
            # draw the text on the image We will be using
            # OpenCV, then draw a bounding box around the
            # text along with the text itself
            # text = "".join(text).strip()
            # cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 0, 255), 2)
            # cv2.putText(img,
            #             text,
            #             (x, y - 10),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1.2, (0, 255, 255), 3)
	
cv2.imwrite(f"Boxes/{filename}", img)

LOTAÇÃO
CPF
CNPJ
COMBUSTÍVEL


True

In [204]:
#generating DF with the results from OCR detection and calculating new columns with the space between words

df = pd.DataFrame.from_dict(results)
df['right'] = df['left']+df['width']
df['space'] = np.nan
df['key'] = np.nan
df['spaceRelativeToHeight'] = np.nan

keyList = []
spaceRelativeToHeightList = []

for i in range(len(df)):
    keyList.append(str(df['page_num'][i])+str(df['block_num'][i])+str(df['par_num'][i])+str(df['line_num'][i]))
    try:
        diff = (df['left'][i+1]-df['right'][i])/df['height'][i]
        if diff<0:
            diff=0
    except:
        diff = np.nan

    spaceRelativeToHeightList.append(diff)

df['containsLower'] = df['text'].str.match(r'(?=.*[a-z])')
df['key'] = keyList
df['spaceRelativeToHeight'] = spaceRelativeToHeightList

In [164]:
#Concatenating nearby words into terms/phrases using new columns calculated

dfWords = df[(df['level']==5)&(df['containsLower']==False)&(df['text']!=' ')].reset_index().drop(columns=['index'])

phrases={}
phrase=''
currentKey = dfWords['key'][0]
left = dfWords['left'][0]
top = dfWords['top'][0]
width = dfWords['width'][0]
height = dfWords['height'][0]

width=0


for i in range(len(dfWords)):
    if (dfWords['spaceRelativeToHeight'][i]<=1) and (dfWords['key'][i]==currentKey):
        phrase+=(dfWords['text'][i]+' ')
        width+=dfWords['width'][i]
    else:
        phrases[phrase.strip()] = {'left':left, 'top':top, 'width':width, 'height':height}

        currentKey = dfWords['key'][i]
        left= dfWords['left'][i]
        top= dfWords['top'][i]
        height= dfWords['height'][i]
        width= dfWords['width'][i]
        phrase = (dfWords['text'][i]+' ')

In [200]:
#Plotting the rectangles over identified terms, using multiplier values 

img2 = cv2.imread('PNGs/DKR8842_2022.pdf_page0.png')
for i in phrases.keys():
    if i in multipliersDict.keys():
        
        cv2.rectangle(img2, (int(phrases[i]['left']),
                             int(phrases[i]['top'])), 
                             (int(phrases[i]['left'] + multipliersDict[i]['width']*(phrases[i]['width'])), 
                              int(phrases[i]['top'] + multipliersDict[i]['height']*(phrases[i]['height']))), (0, 0, 255), 7)
cv2.imwrite("resultBoxes2.png", img2)

True